# 剽窃检测模型

创建了训练和测试数据后，可以定义和训练模型了。在此 notebook 中，你的目标是训练一个二元分类模型，它会根据你提供的特征学习将答案文件标为剽窃文件或非剽窃文件。

此任务将分成以下几个步骤：

* 将数据上传到 S3。
* 定义一个二元分类模型和训练脚本。
* 训练和部署模型。
* 评估部署的分类器并回答关于所采用方法的一些问题。

要完成此 notebook，你需要完成此 notebook 中的所有练习并回答所有问题。
> 所有任务将清晰地标为**练习**，问题都标为**问题**。

你可以尝试不同的分类模型，并选择一个在此数据集上效果最佳的模型。

---

## 将数据上传到 S3

在上个 notebook 中，你应该使用给定剽窃/非剽窃文本数据语料库的特征和类别标签创建了两个文件：`training.csv` 和 `test.csv` 文件。

>以下单元格将加载一些 AWS SageMaker 库并创建一个默认存储桶。创建此存储桶后，你可以将本地存储的数据上传到 S3。

将训练和测试 `.csv` 特征文件保存到本地。你可以在 SageMaker 中运行第二个 notebook“2_Plagiarism_Feature_Engineering”，或者使用 Jupyter Lab 中的上传图标手动将文件上传到此 notebook。然后，你可以使用 `sagemaker_session.upload_data` 将本地文件上传到 S3，并直接指向训练数据的存储位置。

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## 练习：将训练数据上传到 S3

指定在其中保存了 `train.csv` 文件的 `data_dir`。指定一个描述性 `prefix`，指出数据将上传到默认 S3 存储桶的什么位置。最后，通过调用 `sagemaker_session.upload_data` 并传入必要的参数，创建一个指向训练数据的指针。建议参考 [Session 文档](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data)或之前的 SageMaker 代码示例。

你需要上传整个目录。之后，训练脚本将仅访问 `train.csv` 文件。

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/plagiarism_project'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-west-2-270372225889/sagemaker/plagiarism_project


### 测试单元格

测试数据是否已成功上传。以下单元格将输出 S3 存储桶中的内容，如果为空，将抛出错误。你应该看到 `data_dir` 的内容，或许还有一些检查点。如果你看到其中列出了任何其他文件，那么你也许有一些旧的模型文件，你可以通过 S3 控制台删除这些旧文件（不过多余的文件应该不会影响在此 notebook 中开发的模型的性能）。

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

counties/kmeans-2020-04-16-12-43-56-604/output/model.tar.gz
counties/pca-2020-04-15-13-19-51-881/output/model.tar.gz
counties/pca-2020-04-16-12-05-18-083/output/model.tar.gz
creditcard/linear-learner-2020-04-18-04-35-18-741/output/model.tar.gz
creditcard/linear-learner-2020-04-18-04-54-20-427/output/model.tar.gz
linear-learner-2020-04-18-03-41-05-401/output/model.tar.gz
moonproject/sagemaker-pytorch-2020-04-18-11-15-26-852/debug-output/training_job_end.ts
moonproject/sagemaker-pytorch-2020-04-18-11-15-26-852/output/model.tar.gz
sagemaker-pytorch-2020-04-17-13-15-52-328/source/sourcedir.tar.gz
sagemaker-pytorch-2020-04-17-15-35-23-165/source/sourcedir.tar.gz
sagemaker-pytorch-2020-04-18-10-43-48-443/source/sourcedir.tar.gz
sagemaker-pytorch-2020-04-18-11-07-08-631/source/sourcedir.tar.gz
sagemaker-pytorch-2020-04-18-11-11-51-394/source/sourcedir.tar.gz
sagemaker-pytorch-2020-04-18-11-15-26-852/source/sourcedir.tar.gz
sagemaker-pytorch-2020-04-18-11-20-09-593/sourcedir.tar.gz
sagemaker-p

---

# 建模

上传训练数据后，下面定义并训练模型。

你可以决定创建什么类型的模型。对于二元分类任务，你可以选择采用以下三种方法之一：
* 使用内置的分类算法，例如 LinearLearner。
* 定义自定义 Scikit-learn 分类器，可以在[此处](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)找到各个模型的比较情况。
* 定义自定义 PyTorch 神经网络分类器。

你需要测试各种模型并选择最佳模型。我们将根据最终模型的准确率对你的项目打分。
 
---

## 练习：完成训练脚本

为了实现自定义分类器，你需要完成 `train.py` 脚本。我们提供了文件夹 `source_sklearn` 和 `source_pytorch`，其中分别包含自定义 Scikit-learn 模型和 PyTorch 模型的起始代码。每个目录都有一个 `train.py` 训练脚本。要完成此项目，**你只需完成其中一个脚本**，即负责训练最终模型的脚本。

典型的训练脚本会：
* 从指定的目录加载训练数据
* 解析所有的训练和模型超参数（例如神经网络中的节点数，训练周期，等等）
* 实例化你设计的模型，并采用指定的超参数
* 训练该模型
* 最后，保存模型，以便之后托管/部署模型

### 定义和训练模型
我们已经提供了大部分训练脚本。几乎所有任务都位于 `if __name__ == '__main__':` 部分。为了完成 `train.py` 文件，你需要：
1. 导入所需的任何额外库
2. 使用 `parser.add_argument` 定义任何其他模型训练超参数
2. 在 `if __name__ == '__main__':` 部分定义模型
3. 在此部分训练模型

你可以在下面使用 `!pygmentize` 显示现有的 `train.py` 文件。请通读代码，所有任务都标有 `TODO` 注释。 

**注意：如果你选择创建自定义 PyTorch 模型，需要在 `model.py` 文件中定义模型**，并且我们提供了 `predict.py` 文件。如果你选择使用 Scikit-learn，则只需 `train.py` 文件；你可以从 `sklearn` 库中导入一个分类器。

In [7]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib

## TODO: Import any additional libraries you need to define a model


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; set automatically
    # Do not need to change
    par

### 提供的代码

如果你阅读了上述代码，就会发现起始代码包含：
* 模型加载 (`model_fn`) 和保存代码
* 获取 SageMaker 的默认超参数
* 按照名称 `train.csv` 加载训练数据，并提取特征和标签 `train_x` 和 `train_y`

如果你想详细了解如何通过 [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) 或 [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html) 保存模型，请点击提供的链接。

---
# 创建评估器

在 SageMaker 中构建自定义模型时，必须指定入口点。入口点是一个 Python 文件，当模型被训练时，该文件将执行，即你在上面指定的 `train.py` 函数。要在 SageMaker 中运行自定义训练脚本，你需要构建评估器并指定相应的构造函数参数：

* **entry_point**：SageMaker 训练模型和预测时运行的 Python 脚本的路径。
* **source_dir**：训练脚本目录 `source_sklearn` 或 `source_pytorch` 的路径。
* **entry_point**：SageMaker 训练模型和预测时运行的 Python 脚本的路径。
* **source_dir**：训练脚本目录 `train_sklearn` 或 `train_pytorch` 的路径。
* **entry_point**：SageMaker 训练模型时运行的 Python 脚本的路径。
* **source_dir**：训练脚本目录 `train_sklearn` 或 `train_pytorch` 的路径。
* **role**：角色 ARN，在上面已指定。
* **train_instance_count**：训练实例的数量（应该保留为 1）。
* **train_instance_type**：SageMaker 训练实例的类型。注意，因为 Scikit-learn 不提供 GPU 训练原生支持，所以 Sagemaker Scikit-learn 目前不支持在 GPU 实例上训练模型。
* **sagemaker_session**：在 Sagemaker 中训练时使用的会话。
* **hyperparameters**（可选）：作为超参数传递给训练函数的字典 `{'name':value, ..}`。

注意：对于 PyTorch 模型，还有一个可选参数 **framework_version**，你可以将其设为最新的 PyTorch 版本 `1.0`。

## 练习：定义 Scikit-learn 或 PyTorch 评估器

你可以使用以下命令之一导入一个评估器：
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [8]:
from sagemaker.sklearn.estimator import SKLearn

In [9]:
prefix = 'plagiarismproject'
outpath='s3://{}/{}'.format(bucket, prefix)

In [10]:

# your import and estimator code, here
estimator = SKLearn(entry_point='train.py',
                   source_dir='source_sklearn',
                   role=role,
                   train_instance_count=1,
                   train_instance_type='ml.c4.xlarge',
                   output_path=outpath,
                   sagemaker_session=sagemaker_session)

## 练习：训练评估器

使用在 S3 中存储的训练数据训练评估器。代码应该创建一个训练作业，你可以在 SageMaker 控制台中监控该作业。

In [11]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': input_data})


2020-04-23 11:59:56 Starting - Starting the training job...
2020-04-23 11:59:58 Starting - Launching requested ML instances......
2020-04-23 12:00:59 Starting - Preparing the instances for training...
2020-04-23 12:01:44 Downloading - Downloading input data...
2020-04-23 12:02:14 Training - Downloading the training image..2020-04-23 12:02:29,097 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-04-23 12:02:29,100 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-23 12:02:29,110 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-04-23 12:02:29,410 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-04-23 12:02:29,411 sagemaker-containers INFO     Generating setup.cfg
2020-04-23 12:02:29,411 sagemaker-containers INFO     Generating MANIFEST.in
2020-04-23 12:02:29,411 sagemaker-containers INFO     Installing module with the following comm

## 练习：部署训练过的模型

训练之后，部署模型以创建 `predictor`。如果你使用的是 PyTorch 模型，你需要创建一个训练过的 `PyTorchModel`，它会接受训练过的 `<model>.model_data` 作为输入参数，并指向提供的 `source_pytorch/predict.py` 文件作为入口点。

为了部署训练过的模型，你需要使用 `<model>.deploy`，它接受两个参数：
* **initial_instance_count**：部署实例的数量 (1)。
* **instance_type**：部署 SageMaker 实例的类型。

注意：如果你遇到实例错误，可能是因为你选择了错误的训练或部署实例类型。建议参考之前的练习代码，看看我们使用了哪种类型的实例。

In [12]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


---------------!CPU times: user 249 ms, sys: 14.7 ms, total: 263 ms
Wall time: 7min 32s


---
# 评估模型

模型部署后，你可以将模型应用到测试数据上，看看模型的效果如何。

下面提供的单元格会读入测试数据，并假设它存储在本地 `data_dir` 目录下，名称为 `test.csv`。标签和特征是从 `.csv` 文件提取的。

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## 练习：确定模型的准确率

使用部署的 `predictor` 为测试数据预测类别标签。将这些标签与真实标签 `test_y` 进行比较，并计算 0-1 之间的准确率，表示模型分类正确的测试数据所占的比例。你可以使用 [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) 计算准确率。

**要通过此项目，你的模型测试准确率应至少达到 90%。**

In [14]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [15]:
from sklearn.metrics import accuracy_score

In [16]:
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### 问题 1：你的模型生成了多少个假正例和假负例？为何会这样？

In [21]:
test_x.iloc[12,:]

1    0.619792
2    0.100000
3    0.026596
4    0.341584
Name: 12, dtype: float64

**回答：**

没有产生False Positive，产生了1个False Negative。从数据上来看，产生False Negative的ngram参数确实并没有很明显的高。因此模型在该项上产生了误判。但是总体来说是可以接受的。



### 问题 2：你是如何决定要使用什么类型的模型？

**回答：**

这是一个Classification问题。因此可选的包括LinearSVC, Naive Bayes, SGDClassifier, Dicision Tree等等。因为输入基本都为数值，而非bool型，因此排除了Dicision Tree。其他并无特殊偏好。

----
## 练习：清理资源

评估完模型后，记得**删除模型端点**。你可以通过调用 `.delete_endpoint()` 删除端点。你需要在此 notebook 中演示端点已删除。你可以从 AWS 控制台删除任何其他资源，并且在下面找到更多关于删除所有资源的说明。

In [17]:
# uncomment and fill in the line below!
predictor.delete_endpoint()


### 删除 S3 存储桶

完全训练和测试完模型后，你可以删除整个 S3 存储桶。如果你在训练模型之前删除存储桶，需要重新创建 S3 存储桶并将训练数据再次上传到存储桶中。

In [18]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '21AAEE1AD1D2CE37',
   'HostId': 'tFyM1varM7dP13/cd7SC1KjX6YB5KzFxrtLcQP9zTFuGk9DN217eTrCDH0c2dmqbk0VE1XEDmpA=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'tFyM1varM7dP13/cd7SC1KjX6YB5KzFxrtLcQP9zTFuGk9DN217eTrCDH0c2dmqbk0VE1XEDmpA=',
    'x-amz-request-id': '21AAEE1AD1D2CE37',
    'date': 'Thu, 23 Apr 2020 12:15:21 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-pytorch-2020-04-17-15-35-23-165/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-pytorch-2020-04-20-12-08-03-816/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-record-sets/PCA-2020-04-15-13-19-43-199/.amazon.manifest'},
   {'Key': 'sagemaker-pytorch-2020-04-19-03-55-46-417/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-04-18-03-36-50-456/.amazon.manifest'},
   {'Key': 'sagemaker-pytorch-202

### 删除所有模型和实例

当你完全处理完模型，并且**不**需要重新访问此 notebook，你可以根据[这些说明](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)删除所有 SageMaker notebook 实例和模型。在删除此 notebook 实例之前，建议至少下载一个副本并将其保存到本地。

---
## 后续改进建议

此项目还有很多改进或扩展方式，可以拓宽你的知识面，或让此项目更独特。下面是一些建议：
* 训练分类器预测剽窃类别 (1-3)，而不仅仅是剽窃 (1) 或非剽窃 (0)。
* 利用其他更大型的数据集检测此模型能否扩展到其他类型的剽窃行为。
* 利用语言或字符级分析寻找不同（及更多）相似性特征。
* 编写完整的管道函数，它会接受原文和提交的文本，并将提交的文本分类为剽窃或非剽窃文本。
* 使用 API Gateway 和 lambda 函数将模型部署到网络应用上。

这些都只是扩展项目的建议。如果你完成了此 notebook 中的所有练习，你已经完成了一个真实的应用，可以提交项目了。棒棒哒！